# 1. Setup

In [1]:
%load_ext watermark

import glob
import datetime
import numpy as np
import pandas as pd
import xarray as xr

%watermark -iv -co -v

Python implementation: CPython
Python version       : 3.12.4
IPython version      : 8.26.0

conda environment: cpl_ppe_co2

xarray: 2024.6.0
pandas: 2.2.2
numpy : 2.0.1
sys   : 3.12.4 | packaged by conda-forge | (main, Jun 17 2024, 10:23:07) [GCC 12.3.0]



In [12]:
paramfile_dir = '/glade/campaign/asp/djk2120/PPEn11/paramfiles'
paramset_dir = '/glade/u/home/djk2120/ppe_clean/params/parameter_sets'

ex_paramfile = xr.open_dataset(f'{paramfile_dir}/OAAT0001.nc')

oaat_csv = pd.read_csv(f'{paramset_dir}/OAAT_20210408.csv', names=['key', 'name', 'minmax'], header=None)
updated_oaat_csv = pd.read_csv('./params_csv/params_clm5_250415.csv')

dleaf_pft_csv = pd.read_csv('./params_csv/params_clm5_dleaf_250415.csv')
medslope_pft_csv = pd.read_csv('./params_csv/params_clm5_medlynslope_250415.csv')
kmax_pft_csv = pd.read_csv('./params_csv/params_clm5_kmax_250415.csv')

claire_xwalk = pd.read_csv('/glade/u/home/czarakas/coupled_PPE/code/02_set_up_ensemble/CLM5PPE_coupledPPE_crosswalk.csv')

In [13]:
oaat_csv

,key,name,minmax
0,OAAT0000,default,none
1,OAAT0001,taulnir,min
2,OAAT0002,taulnir,max
3,OAAT0003,taulvis,min
4,OAAT0004,taulvis,max
...,...,...,...
396,OAAT0396,jmaxse_sf,max
397,OAAT0397,tpuse_sf,min
398,OAAT0398,tpuse_sf,max
399,OAAT0399,jmax25top_sf,min


In [8]:
dleaf_pft_csv

,pftnum,pftname,dleaf minimum (m),dleaf maximum (m),notes
0,0,not_vegetated,0.000000,0.00000,not_vegetated default is 0; we don't want to v...
1,1,needleleaf_evergreen_temperate_tree,0.000216,0.00108,NaN
2,2,needleleaf_evergreen_boreal_tree,0.000216,0.00108,NaN
3,3,needleleaf_deciduous_boreal_tree,0.000720,0.00360,NaN
4,4,broadleaf_evergreen_tropical_tree,0.008100,0.05670,NaN
...,...,...,...,...,...
74,74,irrigated_switchgrass,0.000162,0.12150,NaN
75,75,tropical_corn,0.000162,0.12150,NaN
76,76,irrigated_tropical_corn,0.000162,0.12150,NaN
77,77,tropical_soybean,0.000162,0.12150,NaN


In [3]:
ex_paramfile

<xarray.Dataset> Size: 105kB
Dimensions:                           (pft: 79, segment: 4, variants: 2)
Coordinates:
    pftname                           (pft) |S40 3kB ...
  * segment                           (segment) |S40 160B b'sunlit           ...
    variantnames                      (variants) object 16B ...
Dimensions without coordinates: pft, variants
Data variables: (12/372)
    C2_liq_Brun89                     float64 8B ...
    FUN_fracfixers                    (pft) float64 632B ...
    a_coef                            float64 8B ...
    a_exp                             float64 8B ...
    a_fix                             (pft) float64 632B ...
    accum_factor                      float64 8B ...
    ...                                ...
    zbedrock_sf                       float64 8B ...
    zlnd                              float64 8B ...
    zsno                              float64 8B ...
    ztopmx                            (pft) float64 632B ...
    crit_onset_gdd_sf                 (pft) float64 632B ...
    ndays_on                          (pft) timedelta64[ns] 632B ...
Attributes:
    history:                    Fri Jun  3 13:40:25 2022: ncks -A -v crit_ons...
    Conventions:                CF-1.0
    title:                      Vegetation (Plant Function Type or PFT) const...
    NCO:                        netCDF Operators version 5.0.3 (Homepage = ht...
    nco_openmp_thread_number:   1
    history_of_appended_files:  Fri Jun  3 13:40:25 2022: Appended file /glad...

In [ ]:
pfinal = [
    'fff',
    'jmaxb0',
    'medlynslope',
    'medlynintercept',
    'maximum_leaf_wetted_fraction',
    'kmax',
    'leafcn',
    'd_max',
    'psi50',
    'cv',
    'dleaf',
    'KCN',
    'FUN_fracfixers',
    'lmrha', # or lmrhd
    # 'ACCLIM_SF',
    # 'SOIL_HYDRO',
]

print('Number of parameters:', len(pfinal))

missing = [param_name for param_name in pfinal if param_name not in updated_oaat_csv['name'].values]
print('Parameters not in the csv file:', missing)

Number of parameters: 14
Parameters not in the csv file: ['KCN']


In [ ]:
updated_oaat_csv.loc[updated_oaat_csv['name'].isin(pfinal), ['name', 'min', 'max']].sort_values(by='min', inplace=False)

,name,min,max
125,FUN_fracfixers,0,1
16,cv,0.005,0.02
46,maximum_leaf_wetted_fraction,0.01,0.5
98,jmaxb0,0.01,0.05
49,fff,0.02,5
80,medlynintercept,1,200000
21,d_max,10,60
108,psi50,30percent,30percent
147,leafcn,30percent,30percent
243,lmrha,50percent,50percent
